In this notebook, I will experiment with the elbow method and the silhouette score to eventually make a plot function which plots the elbow and silhouette graph. Then I can use this in the Population Segmentation Project. First, I make a small 2D dataset with 3 clusters. Afterward I will use the SKlearn K-means algorithm. 

Source: https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/

In [ ]:
#Import libraries
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, silhouette_samples

In [ ]:
#Create the data
x1 = np.array([3, 1, 1, 2, 1, 6, 6, 6, 5, 6, 7, 8, 9, 8, 9, 9, 8])
x2 = np.array([5, 4, 5, 6, 5, 8, 6, 7, 6, 7, 1, 2, 1, 2, 3, 2, 3])

X = np.array(list(zip(x1,x2)))
print("X.shape = {}".format(X.shape))
print("\n", X)


In [ ]:
#Visualize the data
plt.scatter(x1,x2)
plt.title("Dataset")
plt.xlabel("Random variable X1")
plt.ylabel("Random variable X2")
plt.show()

We can see that there are 3 clusters. But this is not always so obvious, e.g. when we have high dimensional data. We are going to calculate 2 metrics for determining the best value for k: distortions and inertia.

Distortion: It is calculated as the average of the squared distances from the cluster centers of the respective clusters. Typically, the Euclidean distance metric is used.

Inertia: It is the sum of squared distances of samples to their closest cluster center.

In [ ]:
inertias = []
K = range(1,12)

sil_scores = []

#In this case we loop over one value of k to see what is happening
for k in K:
    
    print("\nK = {}".format(k))
    
    #Building and fitting the model
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    
    #Obtain the cluster centers
    cluster_centers = kmeans.cluster_centers_
    print("\ncluster_centers = \n{}".format(cluster_centers))
    
    #Calculate the distances from each point to the cluster center
    distance = cdist(X, cluster_centers, "euclidean")
    print("\ndistance = \n{}".format(distance))
    #Note: The first column in distance represents the distance of the all datapoints to the first cluster center and so on.  
    
    #Determine the minimum distance. This takes the minimum 
    min_distance = np.min(distance, axis=1)
    print("\nmin_distance = \n{}".format(min_distance))
    #Note: These are the distances of each datapoint to the closest cluster
    
    #Calculate the inertia, which is the sum of all distances (from the datapoint to closest cluster center) divided by the total datapoints
    inertia = sum(np.square(min_distance)) #/ X.shape[0]
    print("\nInertia = {}".format(inertia))
    
    #Append the inertia to the list
    inertias.append(inertia)
    
    #To calculate the silhouette score we need the clusters to which the datapoints are assigned
    cluster = np.argmin(distance, axis=1)
    print("\nClusters = \n{}".format(cluster))
    
    #Calculate the average silhouette score, This gives a perspective into the density and separation 
    #of the formed clusters. Because the silhouette score only works for k=2 or more, we introduce and if statement 
    if k >= 2: 
        sil_avg = silhouette_score(X, cluster)
        print("\nFor k = {} , The average silhouette_score is : {}".format(k, sil_avg))
        
        #Compute the silhouette score for each of the samples
        sample_sil_values = silhouette_samples(X, cluster)
        
        #Append to sil_scores list.
        sil_scores.append(sil_avg)
        print("sil_scores = {}".format(sil_scores))
    
    else:
        print("Number of clusters is one. No silhouette score available.")
    
    print("\n----------------------------------------------------------------------------------------------\n")
    
print("\nInertias of all the K's = \n{}".format(inertias))
print("\nSilhouette scores of all the K's = \n{}".format(sil_scores))
print("")
#Plot elbow graph with inertia 
plt.plot(K, inertias, "bx-")
plt.title("Elbow graph with inertias")
plt.xlabel("K")
plt.ylabel("Inertia")
plt.show()

#Plot silhouette scores graph
if K[0] >= 2:
    plt.plot(K, sil_scores, "bx-")
    plt.title("Silhouette graph")
    plt.xlabel("K")
    plt.ylabel("Silhouette score")
    plt.show()
else:
    plt.plot(K[1:], sil_scores, "bx-")
    plt.title("Silhouette graph")
    plt.xlabel("K")
    plt.ylabel("Silhouette score")
    plt.show()
    

This is what np.min(X, axis=1) does: 

In [ ]:
y1 = np.array([1,0.5,-1])
y2 = np.array([0,1,0])

Y = np.array(list(zip(y1, y2))).reshape(len(y1), 2)
print("Y = \n{}".format(Y))

#Take the minimin value of y
min_Y = np.min(Y)
print("\nmin_Y = {}".format(min_Y))

#Take the column with the lowest value
min_Y_column = np.min(Y, axis=1)
print("\nmin_Y_column = {}".format(min_Y_column))

#If we specify axis=1, it goes through all rows and compares the values of the columns and takes the min. See print below.

Thus what happens is that, for each datapoint, the distances to each cluster center gets calculated and we take the minimum distance: this gives the cluster to which the datapoint belongs.

Now that we see everything works, lets make a function which we can import to our main notebook to perform this analyses on the population segmentation data. 

In [ ]:
def plot_elbow_silhouette(X, K, cluster_centers):
    
    """
    X the data (counties_transformed which has 7 columns)
    K should be a range()
    cluster_centers: a 2D numpy array containing the cluster centers 
    """
    
    #Initialize some lists which we will append to later
    distortions = []
    inertias = []
    sil_scores = []
    
    #In this case we loop over one value of k to see what is happening
    for k in K:

        print("\nK = {}".format(k))

        #Calculate the distances from each point to the cluster center
        distance = cdist(X, cluster_centers, "euclidean")
        print("\ndistance = \n{}".format(distance))
        #Note: The first column in distance represents the distance of the all datapoints to the first cluster center and so on.  

        #Determine the minimum distance. This takes the minimum 
        min_distance = np.min(distance, axis=1)
        print("\nmin_distance = \n{}".format(min_distance))
        #Note: These are the distances of each datapoint to the closest cluster

        #Calculate the distortion, which is the sum of all distances (from the datapoint to closest cluster center) divided by the total datapoints
        distortion = sum(np.square(min_distance)) #/ X.shape[0]
        print("\nDistortion = {}".format(distortion))

        #Append the distortion to the list
        distortions.append(distortion)

        #To calculate the silhouette score we need the clusters to which the datapoints are assigned
        cluster = np.argmin(distance, axis=1)
        print("\nClusters = \n{}".format(cluster))

        #Calculate the average silhouette score, This gives a perspective into the density and separation 
        #of the formed clusters. Because the silhouette score only works for k=2 or more, we introduce and if statement 
        if k >= 2: 
            sil_avg = silhouette_score(X, cluster)
            print("\nFor k = {} , The average silhouette_score is : {}".format(k, sil_avg))

            #Compute the silhouette score for each of the samples
            sample_sil_values = silhouette_samples(X, cluster)

            #Append to sil_scores list.
            sil_scores.append(sil_avg)
            print("sil_scores = {}".format(sil_scores))

        else:
            print("Number of clusters is one. No silhouette score available.")

        print("\n----------------------------------------------------------------------------------------------\n")

    print("\nDistortions of all the K's = \n{}".format(distortions))
    print("\nSilhouette scores of all the K's = \n{}".format(sil_scores))
    print("")
    #Plot elbow graph with inertia 
    plt.plot(K, distortions, "bx-")
    plt.title("Elbow graph with inertias")
    plt.xlabel("K")
    plt.ylabel("Inertia")
    plt.show()

    #Plot silhouette scores graph
    if K[0] >= 2:
        plt.plot(K, sil_scores, "bx-")
        plt.title("Silhouette graph")
        plt.xlabel("K")
        plt.ylabel("Silhouette score")
        plt.show()
    else:
        plt.plot(K[1:], sil_scores, "bx-")
        plt.title("Silhouette graph")
        plt.xlabel("K")
        plt.ylabel("Silhouette score")
        plt.show()
    

In [ ]:
def inertia_silhouette_score(X, K):
    
    """
    X the data (counties_transformed which has 7 columns)
    K should be a range()
    cluster_centers: a 2D numpy array containing the cluster centers 
    """

    print("\nK = {}".format(K))
    
    #Building and fitting the model
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    
    #Obtain the cluster centers
    cluster_centers = kmeans.cluster_centers_
    print("\ncluster_centers = \n{}".format(cluster_centers))

    #Calculate the distances from each point to the cluster center
    distance = cdist(X, cluster_centers, "euclidean")
    print("\ndistance = \n{}".format(distance))
    #Note: The first column in distance represents the distance of the all datapoints to the first cluster center and so on.  

    #Determine the minimum distance. This takes the minimum 
    min_distance = np.min(distance, axis=1)
    print("\nmin_distance = \n{}".format(min_distance))
    #Note: These are the distances of each datapoint to the closest cluster

    #Calculate the inertia, which is the sum of squared distances (from the datapoint to closest cluster center) divided by the total datapoints
    inertia = sum(np.square(min_distance)) #/ X.shape[0]
    print("\nInertia = {}".format(inertia))

    #Append the inertia to the list
    inertias.append(inertia)

    #To calculate the silhouette score we need the clusters to which the datapoints are assigned
    cluster = np.argmin(distance, axis=1)
    print("\nClusters = \n{}".format(cluster))

    #Calculate the average silhouette score, This gives a perspective into the density and separation 
    #of the formed clusters. Because the silhouette score only works for k=2 or more, we introduce and if statement 
    if K >= 2: 
        sil_avg = silhouette_score(X, cluster)
        print("\nFor k = {} , The average silhouette_score is : {}".format(K, sil_avg))

        #Compute the silhouette score for each of the samples
        sample_sil_values = silhouette_samples(X, cluster)

        #Append to sil_scores list.
        sil_scores.append(sil_avg)
        print("sil_scores = {}".format(sil_scores))

    else:
        print("Number of clusters is one. No silhouette score available.")

    print("\n----------------------------------------------------------------------------------------------\n")

    print("\nInertias of all the K's = \n{}".format(inertias))
    print("\nSilhouette scores of all the K's = \n{}".format(sil_scores))
    print("")
    
    return inertias, sil_scores

In [ ]:
inertia, silhouette = inertia_silhouette_score(X, K=3)